In [1]:
import pandas as pd
import numpy as np
import re 
import random

# load data

In [2]:
# df = pd.read_csv('record_11Oct.csv', encoding='utf8')
df = pd.read_excel('base/4process_25Nov.xlsx')

In [3]:
df.head(3)

,STT,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,TT_Thời gian sử dụng,TT_Note,KL_Nội dung hỗ trợ,KL_flow,KL_Issue,KL_Success,KL_Warning,KL_Consideration,KL_Đánh giá NV + đề xuất
0,1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,31.0,-,Thanh lý hợp đồng internet,n12=s1>ds1\s2>ds3\s3>dx5\nn34=s1>hd5,NaN,NaN,NaN,NaN,NaN
1,2,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,31.0,KH say ke,Thanh lý hợp đồng internet,n00=s1>ds1+dx4\nn10=s1>ds3\s2>ds1\s3>ll3+dx5\n...,NaN,NaN,NaN,NaN,NaN
2,3,0973641746,HNAAG3366,Giga,Hà Nội,Nam,35 - 65,Có hiểu biết,-,30.0,-,Tạm ngưng hợp đồng internet,n00=s1>gt1+ds1\nn22=s1>ll3+dx4\s2>dx5\nn34=s1>...,NaN,NaN,NaN,NaN,NaN


In [4]:
df['KL_flow'].isna().sum()

0

In [5]:
df = df.dropna(subset=['KL_flow'])
df['KL_flow'].isna().sum()

0

In [6]:
df.columns

Index(['STT', 'DV_Số điện thoại', 'DV_Số hợp đồng', 'DV_Dịch vụ', 'TT_Vùng',
       'TT_Giới tính', 'TT_Nhóm tuổi', 'TT_Mức độ am hiểu',
       'TT_Mục đích sử dụng', 'TT_Thời gian sử dụng', 'TT_Note',
       'KL_Nội dung hỗ trợ', 'KL_flow', 'KL_Issue', 'KL_Success', 'KL_Warning',
       'KL_Consideration', 'KL_Đánh giá NV + đề xuất'],
      dtype='object')

In [7]:
# add necessary column to do
df = df.assign(
    **{col: None for col in [
        'Issue Code', 'Expand Issue', 'Indicate Issue', 'Quote Issue', 'Indicate Success', 'Detail Success', 'Quote Success',
        'Indicate Warning 1', 'Detail Warning 1', 'Quote Warning 1', 
        'Indicate Warning 2', 'Detail Warning 2', 'Quote Warning 2', 
        'Indicate Warning 3', 'Detail Warning 3', 'Quote Warning 3',
        'Indicate Consideration 1', 'Detail Consideration 1', 'Quote Consideration 1',
        'Indicate Consideration 2', 'Detail Consideration 2', 'Quote Consideration 2',
        'Indicate Consideration 3', 'Detail Consideration 3', 'Quote Consideration 3'
    ]}
)

In [8]:
df[df['KL_Nội dung hỗ trợ'].isna()]

,STT,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,TT_Thời gian sử dụng,...,Quote Warning 3,Indicate Consideration 1,Detail Consideration 1,Quote Consideration 1,Indicate Consideration 2,Detail Consideration 2,Quote Consideration 2,Indicate Consideration 3,Detail Consideration 3,Quote Consideration 3


# Index split into smaller components

In [9]:
def split_and_index_kl_flow(df):

    df['KL_flow'] = df['KL_flow'].astype(str)
    
    processed_data = []
    
    for idx, row in df.iterrows():
        stt = row['STT']
        kl_flow = row['KL_flow']
        
        flow_lines = kl_flow.split('\n')
        
        for sub_id, flow in enumerate(flow_lines, start=1):
            new_row = row.copy()  # Sao chép hàng để giữ nguyên các cột khác
            new_row['sub_id'] = f"{stt}.{sub_id}"  # Tạo sub_id bằng cách ghép STT và sub_id
            new_row['KL_flow'] = flow  # Gán dòng hiện tại vào KL_flow
            processed_data.append(new_row)

    processed_df = pd.DataFrame(processed_data)
    
    # Sắp xếp lại các cột: Đưa sub_id ngay sau STT
    cols = list(processed_df.columns)
    cols.insert(1, cols.pop(cols.index('sub_id')))  # Đưa sub_id lên vị trí thứ hai
    processed_df = processed_df[cols]
    
    processed_df.reset_index(drop=True, inplace=True)
    
    return processed_df

In [10]:
df = split_and_index_kl_flow(df)

In [11]:
df.head(3)

,STT,sub_id,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,...,Quote Warning 3,Indicate Consideration 1,Detail Consideration 1,Quote Consideration 1,Indicate Consideration 2,Detail Consideration 2,Quote Consideration 2,Indicate Consideration 3,Detail Consideration 3,Quote Consideration 3
0,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
1,1,1.2,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
2,2,2.1,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,None,None,None,None,None,None,None,None,None,None


# get indicate and detail

In [12]:
def process_kl_flow(df):
    temp_results = []

    for index, row in df.iterrows():
        KL_flow = row['KL_flow']
        
        issue_code = None
        expand_issue = None
        indicate_issue = None
    
        indicate_success = None
        detail_success = None

        indicate_warning = [None, None, None]
        detail_warning = [None, None, None]
        
        indicate_consideration = [None, None, None]
        detail_consideration = [None, None, None]

        temp_bhe = KL_flow.split('=')[0].strip()
        
        if '[' in temp_bhe:
            expand_issue = '[' + str(temp_bhe.split('[', 1)[1].strip())

        kl_flow = KL_flow.replace(str(expand_issue), '')

        issue_code = kl_flow.split('=', 1)[0].strip()

        if kl_flow:
            if '#' in kl_flow:
                issue_code = kl_flow.split('#')[0].strip()
                indicate_issue = issue_code + '#'
             
            # bhe behind the equal
            if '=' in kl_flow:
                bhe = kl_flow.split('=')[1].strip()
            # else:
            #     bhe = None

            if bhe:
                if "'" in bhe:
                    issue_code = kl_flow.split("=")[0].strip().replace("#", "")
                    temp_string_font_symbol = bhe.split("'")[0].strip()
                    indicate_success = temp_string_font_symbol.rsplit("\\", 1)[-1].strip()
                    template = indicate_success + "'"
                    temp_var = bhe.split(template)[1]
                    detail_success = temp_var.split("\\")[0].replace(">", "").strip()
                            
                if "^" in bhe:
                    issue_code = kl_flow.split("=")[0].strip().replace("#", "")
                    count = kl_flow.count("^")
                    components = bhe.split('^')
        
                    for i in range(count):
                        indicate_component = components[i].strip()
                        
                        temp_var = indicate_component.rsplit("\\", 1)[-1].strip()
                        indicate_consideration[i] = temp_var
                
                    for i, item in enumerate(indicate_consideration):
                        if item is not None:
                            template = item + "^"
                            split_at_con = bhe.split(template, 1)[-1].strip()
                            detail_temp = split_at_con.split("\\")[0].replace(">", "").strip()
                            detail_consideration[i] = detail_temp

                if "!" in bhe:
                    issue_code = kl_flow.split("=")[0].strip().replace("#", "")
                    count = kl_flow.count("!")
                    components = bhe.split('!')
                    for i in range(count):
                        indicate_component = components[i].strip()
                        
                        temp_var = indicate_component.rsplit("\\", 1)[-1].strip()
                        indicate_warning[i] = temp_var
                            
                    for i, item in enumerate(indicate_warning):
                        if item is not None:
                            template = item + "!"
                            split_at_con = bhe.split(template, 1)[-1].strip()
                            detail_temp = split_at_con.split("\\")[0].replace(">", "").strip()
                            detail_warning[i] = detail_temp

        temp_results.append({
            'index': index,
            'Issue Code': issue_code,
            'Indicate Issue': indicate_issue,
            'Indicate Success': indicate_success,
            'Expand Issue': expand_issue,
            'Detail Success': detail_success,
            'Indicate Warning 1': indicate_warning[0],
            'Detail Warning 1': detail_warning[0],
            'Indicate Warning 2': indicate_warning[1],
            'Detail Warning 2': detail_warning[1],
            'Indicate Warning 3': indicate_warning[2],
            'Detail Warning 3': detail_warning[2],
            'Indicate Consideration 1': indicate_consideration[0],
            'Detail Consideration 1': detail_consideration[0],
            'Indicate Consideration 2': indicate_consideration[1],
            'Detail Consideration 2': detail_consideration[1],
            'Indicate Consideration 3': indicate_consideration[2],
            'Detail Consideration 3': detail_consideration[2]
        })

        # print(f"Processed index {index}:")
        # print(f"Flow {kl_flow}:")
        # print(f"  Issue Code: {issue_code}, Indicate Issue: {indicate_issue}, Expand Issue: {expand_issue}")
        # print(f"  Indicate Success: {indicate_success}, Detail Success: {detail_success}")
        # print(f"  Indicate Warning: {indicate_warning}, Detail Warning: {detail_warning}")
        # print(f"  Indicate Consideration: {indicate_consideration}, Detail Consideration: {detail_consideration}\n")

    for result in temp_results:
        index = result['index']
        df.loc[index, 'Issue Code'] = result['Issue Code']
        df.loc[index, 'Expand Issue'] = result['Expand Issue']
        df.loc[index, 'Indicate Issue'] = result['Indicate Issue']
        df.loc[index, 'Indicate Success'] = result['Indicate Success']
        df.loc[index, 'Detail Success'] = result['Detail Success']
        df.loc[index, 'Indicate Warning 1'] = result['Indicate Warning 1']
        df.loc[index, 'Detail Warning 1'] = result['Detail Warning 1']
        df.loc[index, 'Indicate Warning 2'] = result['Indicate Warning 2']
        df.loc[index, 'Detail Warning 2'] = result['Detail Warning 2']
        df.loc[index, 'Indicate Warning 3'] = result['Indicate Warning 3']
        df.loc[index, 'Detail Warning 3'] = result['Detail Warning 3']
        df.loc[index, 'Indicate Consideration 1'] = result['Indicate Consideration 1']
        df.loc[index, 'Detail Consideration 1'] = result['Detail Consideration 1']
        df.loc[index, 'Indicate Consideration 2'] = result['Indicate Consideration 2']
        df.loc[index, 'Detail Consideration 2'] = result['Detail Consideration 2']
        df.loc[index, 'Indicate Consideration 3'] = result['Indicate Consideration 3']
        df.loc[index, 'Detail Consideration 3'] = result['Detail Consideration 3']

    return df

In [13]:
df = process_kl_flow(df)

In [14]:
df.shape

(1239, 44)

In [15]:
df.head(3)

,STT,sub_id,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,...,Quote Warning 3,Indicate Consideration 1,Detail Consideration 1,Quote Consideration 1,Indicate Consideration 2,Detail Consideration 2,Quote Consideration 2,Indicate Consideration 3,Detail Consideration 3,Quote Consideration 3
0,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
1,1,1.2,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
2,2,2.1,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,None,None,None,None,None,None,None,None,None,None


# get quote base on indicate

In [16]:
def get_quote(df):
    for index, row in df.iterrows():
        # Initialize quotes for each category
        quote_issue = None
        quote_success = None
        
        quote_warning = [None, None, None]    
        quote_consideration = [None, None, None]

        # Process quote issue
        if pd.notna(row['Indicate Issue']):
            flag = row['Indicate Issue']
            temp_quote = row['KL_Issue']
            if pd.notna(temp_quote):
                try:
                    temp_quote_2 = temp_quote.split(flag, 1)[1]
                    temp_quote_3 = temp_quote_2.split("\n", 1)[1]
                    quote_issue = temp_quote_3.split("\n", 1)[0] if "\n" in temp_quote_3 else temp_quote_3
                except IndexError:
                    quote_issue = None

        # Process quote success
        if pd.notna(row['Indicate Success']):
            flag = row['Indicate Success']
            temp_quote = row['KL_Success']
            if pd.notna(temp_quote):
                try:
                    temp_quote_2 = temp_quote.split(flag, 1)[1]
                    temp_quote_3 = temp_quote_2.split("\n", 1)[1]
                    quote_success = temp_quote_3.split("\n", 1)[0] if "\n" in temp_quote_3 else temp_quote_3
                except IndexError:
                    quote_success = None

        # Process quote consideration for 3 possible entries
        for i in range(3):
            if pd.notna(row[f'Indicate Consideration {i+1}']):
                flag = row[f'Indicate Consideration {i+1}']
                temp_quote = row['KL_Consideration']
                if pd.notna(temp_quote):
                    try:
                        temp_quote_2 = temp_quote.split(flag, 1)[1]
                        temp_quote_3 = temp_quote_2.split("\n", 1)[1]
                        quote_consideration[i] = temp_quote_3.split("\n", 1)[0] if "\n" in temp_quote_3 else temp_quote_3
                    except IndexError:
                        quote_consideration[i] = None

        # Process quote warning for 3 possible entries
        for i in range(3):
            if pd.notna(row[f'Indicate Warning {i+1}']):
                flag = row[f'Indicate Warning {i+1}']
                temp_quote = row['KL_Warning']
                if pd.notna(temp_quote):
                    try:
                        temp_quote_2 = temp_quote.split(flag, 1)[1]
                        temp_quote_3 = temp_quote_2.split("\n", 1)[1]
                        quote_warning[i] = temp_quote_3.split("\n", 1)[0] if "\n" in temp_quote_3 else temp_quote_3
                    except IndexError:
                        quote_warning[i] = None

        # Assign quotes back to DataFrame
        df.loc[index, 'Quote Issue'] = quote_issue
        df.loc[index, 'Quote Success'] = quote_success

        df.loc[index, 'Quote Warning 1'] = quote_warning[0]
        df.loc[index, 'Quote Warning 2'] = quote_warning[1]
        df.loc[index, 'Quote Warning 3'] = quote_warning[2]

        df.loc[index, 'Quote Consideration 1'] = quote_consideration[0]
        df.loc[index, 'Quote Consideration 2'] = quote_consideration[1]
        df.loc[index, 'Quote Consideration 3'] = quote_consideration[2]

    return df

In [17]:
df = get_quote(df)

In [18]:
df

,STT,sub_id,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,...,Quote Warning 3,Indicate Consideration 1,Detail Consideration 1,Quote Consideration 1,Indicate Consideration 2,Detail Consideration 2,Quote Consideration 2,Indicate Consideration 3,Detail Consideration 3,Quote Consideration 3
0,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
1,1,1.2,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
2,2,2.1,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,None,None,None,None,None,None,None,None,None,None
3,2,2.2,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,None,None,None,None,None,None,None,None,None,None
4,2,2.3,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,683,683.2,907780518,SGJ099035,Sky,Hồ Chí Minh,Nam,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
1235,683,683.3,907780518,SGJ099035,Sky,Hồ Chí Minh,Nam,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
1236,684,684.1,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,None,None,None,None,None,None,None,None,None,None
1237,684,684.2,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,None,None,None,None,None,None,None,None,None,None


# extract level 1

In [19]:
df.rename(columns={'STT': 'case_id', 'sub_id': 'flow_id', 'KL_flow':'Flow'}, inplace=True)

In [20]:
df

,case_id,flow_id,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,...,Quote Warning 3,Indicate Consideration 1,Detail Consideration 1,Quote Consideration 1,Indicate Consideration 2,Detail Consideration 2,Quote Consideration 2,Indicate Consideration 3,Detail Consideration 3,Quote Consideration 3
0,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
1,1,1.2,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
2,2,2.1,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,None,None,None,None,None,None,None,None,None,None
3,2,2.2,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,None,None,None,None,None,None,None,None,None,None
4,2,2.3,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,683,683.2,907780518,SGJ099035,Sky,Hồ Chí Minh,Nam,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
1235,683,683.3,907780518,SGJ099035,Sky,Hồ Chí Minh,Nam,18 - 35,Có hiểu biết,-,...,None,None,None,None,None,None,None,None,None,None
1236,684,684.1,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,None,None,None,None,None,None,None,None,None,None
1237,684,684.2,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,None,None,None,None,None,None,None,None,None,None


In [21]:
df = df.dropna(subset=['Flow'])

In [22]:
 df.columns

Index(['case_id', 'flow_id', 'DV_Số điện thoại', 'DV_Số hợp đồng',
       'DV_Dịch vụ', 'TT_Vùng', 'TT_Giới tính', 'TT_Nhóm tuổi',
       'TT_Mức độ am hiểu', 'TT_Mục đích sử dụng', 'TT_Thời gian sử dụng',
       'TT_Note', 'KL_Nội dung hỗ trợ', 'Flow', 'KL_Issue', 'KL_Success',
       'KL_Warning', 'KL_Consideration', 'KL_Đánh giá NV + đề xuất',
       'Issue Code', 'Expand Issue', 'Indicate Issue', 'Quote Issue',
       'Indicate Success', 'Detail Success', 'Quote Success',
       'Indicate Warning 1', 'Detail Warning 1', 'Quote Warning 1',
       'Indicate Warning 2', 'Detail Warning 2', 'Quote Warning 2',
       'Indicate Warning 3', 'Detail Warning 3', 'Quote Warning 3',
       'Indicate Consideration 1', 'Detail Consideration 1',
       'Quote Consideration 1', 'Indicate Consideration 2',
       'Detail Consideration 2', 'Quote Consideration 2',
       'Indicate Consideration 3', 'Detail Consideration 3',
       'Quote Consideration 3'],
      dtype='object')

# split solution & protocol

In [23]:
def extract_solution(df):
    processed_data = []
    
    for index, row in df.iterrows():
        case_id = row['case_id']
        flow_id = row['flow_id']
        flow_value = row['Flow']

        # Split the 'Flow' value based on '=' and then '\'
        solution_lines = flow_value.split('=')[1] if '=' in flow_value else flow_value
        solution_lines = solution_lines.split('\\')

        # Iterate through each solution line
        for solution_id, item in enumerate(solution_lines, start=1):
            solution = item.split('>')[0]
            protocols = item.split('>')[1] if '>' in item else ''

            protocols = "-" + str(protocols) + "-"
            
            # Process protocols
            protocols_list = re.findall(r'[^:+-]+|[:+-]', protocols)

            # Check for symbols (^, !, ') and split accordingly
            if '^' in solution:
                just_a_solution = solution.split('^')[0]
                solution_result = 'Cân nhắc'
            elif '!' in solution:
                just_a_solution = solution.split('!')[0]
                solution_result = 'Lưu ý'
            elif "'" in solution:
                just_a_solution = solution.split("'")[0]
                solution_result = "Thành công"
            else:
                just_a_solution = solution  # No symbol found, keep the solution as is
                solution_result = "Không thành công"  # No symbol means no result     

            # Add the main solution row
            new_row = row.copy()  # Copy the row to keep other columns the same
            new_row['solution_id'] = f"{flow_id}.{solution_id}" 
            new_row['Solution'] = just_a_solution
            new_row['Solution Result'] = solution_result

            # Append the main solution row to processed_data
            processed_data.append(new_row)
            
            index_id = 1
            for protocol_id, protocol in enumerate(protocols_list, start=1):
                if protocol not in ['-', '+', ':']:
                    protocol_row = new_row.copy()
            
                    # Xác định giá trị cho previous_protocol
                    previous_protocol = protocols_list[protocol_id - 2] if protocol_id > 1 else 'None'
                    # Xác định giá trị cho next_protocol
                    next_protocol = protocols_list[protocol_id] if protocol_id < len(protocols_list) else 'None'
            
                    protocol_row['protocol_id'] = f"{new_row['solution_id']}.{index_id}"
                    protocol_row['Forward Cohesion'] = previous_protocol
                    protocol_row['Protocol'] = protocol.split('[', 1)[0]
                    
                    # Check if there's a detail in brackets
                    protocol_detail_split = protocol.split('[', 1)
                    if len(protocol_detail_split) > 1:
                        protocol_row['Protocol Detail'] = "[" + str(protocol_detail_split[1])        
                    else:
                        protocol_row['Protocol Detail'] = ""  # Or any default value you prefer
                        
                    protocol_row['Backward Cohesion'] = next_protocol
                    
                    # Append the protocol row to processed_data
                    processed_data.append(protocol_row)
                    index_id += 1


    # Convert the processed data into a new DataFrame
    processed_df = pd.DataFrame(processed_data)
    
    # Reorder columns: Insert sub_id right after STT
    cols = list(processed_df.columns)
    processed_df = processed_df[cols]
    
    # Reset the index
    processed_df.reset_index(drop=True, inplace=True)
                    
    return processed_df

In [24]:
df = extract_solution(df)
df = df.dropna(subset=['Forward Cohesion', 'Protocol', 'Backward Cohesion'], how='all')

In [25]:
df

,case_id,flow_id,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,...,Detail Consideration 3,Quote Consideration 3,solution_id,Solution,Solution Result,protocol_id,Forward Cohesion,Protocol,Protocol Detail,Backward Cohesion
1,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,1.1.1,s1,Không thành công,1.1.1.1,-,ds1,,-
3,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,1.1.2,s2,Không thành công,1.1.2.1,-,ds3,,-
5,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,1.1.3,s3,Không thành công,1.1.3.1,-,dx5,,-
7,1,1.2,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,None,1.2.1,s1,Không thành công,1.2.1.1,-,hd5,,-
9,2,2.1,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,None,None,2.1.1,s1,Không thành công,2.1.1.1,-,ds1,,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5869,684,684.2,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,None,None,684.2.2,s2,Không thành công,684.2.2.1,-,ll3,,+
5870,684,684.2,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,None,None,684.2.2,s2,Không thành công,684.2.2.2,+,ll5,,-
5872,684,684.3,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,None,None,684.3.1,s1,Cân nhắc,684.3.1.1,-,dx7,,+
5873,684,684.3,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,None,None,684.3.1,s1,Cân nhắc,684.3.1.2,+,gt3,,+


# label them

In [26]:
df.columns

Index(['case_id', 'flow_id', 'DV_Số điện thoại', 'DV_Số hợp đồng',
       'DV_Dịch vụ', 'TT_Vùng', 'TT_Giới tính', 'TT_Nhóm tuổi',
       'TT_Mức độ am hiểu', 'TT_Mục đích sử dụng', 'TT_Thời gian sử dụng',
       'TT_Note', 'KL_Nội dung hỗ trợ', 'Flow', 'KL_Issue', 'KL_Success',
       'KL_Warning', 'KL_Consideration', 'KL_Đánh giá NV + đề xuất',
       'Issue Code', 'Expand Issue', 'Indicate Issue', 'Quote Issue',
       'Indicate Success', 'Detail Success', 'Quote Success',
       'Indicate Warning 1', 'Detail Warning 1', 'Quote Warning 1',
       'Indicate Warning 2', 'Detail Warning 2', 'Quote Warning 2',
       'Indicate Warning 3', 'Detail Warning 3', 'Quote Warning 3',
       'Indicate Consideration 1', 'Detail Consideration 1',
       'Quote Consideration 1', 'Indicate Consideration 2',
       'Detail Consideration 2', 'Quote Consideration 2',
       'Indicate Consideration 3', 'Detail Consideration 3',
       'Quote Consideration 3', 'solution_id', 'Solution', 'Solution Result

In [27]:
nn = pd.read_excel('base/label.xlsx', sheet_name='nguyên nhân')

issue_map = dict(zip(nn['Tên nguyên nhân'], nn['Mã nhóm nguyên nhân']))
df['Issue_category'] = df['Issue Code'].map(issue_map)

In [28]:
nn

,Nhóm nguyên nhân,Mã nhóm nguyên nhân,Tên nguyên nhân,Nguyên nhân
0,Nguyên nhân liên quan đến quyết định chưa rõ ràng,G0,n00,Khách hàng yêu cầu hỗ trợ nhưng chưa rõ lý do ...
1,Nguyên nhân liên quan đến quyết định chưa rõ ràng,G0,n01,Khách hàng không/chưa chia sẻ nguyên nhân
2,Nguyên nhân liên quan đến địa điểm,G1,n10,"Khách hàng chuyển đến địa chỉ mới, chưa xác đị..."
3,Nguyên nhân liên quan đến địa điểm,G1,n11,Khách hàng chuyển đến địa điểm mới và địa chỉ ...
4,Nguyên nhân liên quan đến địa điểm,G1,n12,Khách hàng chuyển đến địa chỉ mới có sẵn đường...
...,...,...,...,...
56,Nguyên nhân bổ sung có thể phát sinh,G6,n66,Khách hàng cảm thấy không hài lòng với quá trì...
57,Nguyên nhân bổ sung có thể phát sinh,G6,n68,Khách hàng đã nhượng lại đường truyền cho ngườ...
58,Nguyên nhân bổ sung có thể phát sinh,G6,n69,Bên mới hoặc cũ cân nhắc nhu cầu sử dụng lại đ...
59,Nguyên nhân bổ sung có thể phát sinh,G6,n610,"Khách hàng ấn nhẩm, nhờ hủy dịch vụ"


In [29]:
df.head(3)

,case_id,flow_id,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,...,Quote Consideration 3,solution_id,Solution,Solution Result,protocol_id,Forward Cohesion,Protocol,Protocol Detail,Backward Cohesion,Issue_category
1,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,1.1.1,s1,Không thành công,1.1.1.1,-,ds1,,-,G1
3,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,1.1.2,s2,Không thành công,1.1.2.1,-,ds3,,-,G1
5,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,None,1.1.3,s3,Không thành công,1.1.3.1,-,dx5,,-,G1


In [30]:
dx = pd.read_excel('base/label.xlsx', sheet_name='đề xuất')
ll = pd.read_excel('base/label.xlsx', sheet_name='lý lẽ')
hd = pd.read_excel('base/label.xlsx', sheet_name='hướng dẫn')
ds = pd.read_excel('base/label.xlsx', sheet_name='đào sâu')
gt = pd.read_excel('base/label.xlsx', sheet_name='giải thích')
pg = pd.read_excel('base/label.xlsx', sheet_name='protocol group')
nn = pd.read_excel('base/label.xlsx', sheet_name='nguyên nhân')

label = pd.concat([dx, ll, hd, ds, gt], ignore_index=True)

In [31]:
df['Mô tả protocol'] = df['Protocol'].str.split("[", n=1).str[0]
label['Mô tả'] = label['Mô tả'].str.split("[", n=1).str[0]

issue_protocol = dict(zip(label['Mã'], label['Mô tả']))
df['Mô tả protocol'] = df['Mô tả protocol'].map(issue_protocol)

issue_des = dict(zip(nn['Tên nguyên nhân'], nn['Nguyên nhân']))
df['Mô tả nguyên nhân'] = df['Issue Code'].map(issue_des)

group_issue_des = dict(zip(nn['Mã nhóm nguyên nhân'], nn['Nhóm nguyên nhân']))
df['Mô tả nhóm nguyên nhân'] = df['Issue_category'].map(group_issue_des)

protocol_group = dict(zip(pg['Mã'], pg['Protocol Group']))
df['Nhóm Protocol'] = df['Protocol'].map(protocol_group)

In [32]:
df

,case_id,flow_id,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,...,protocol_id,Forward Cohesion,Protocol,Protocol Detail,Backward Cohesion,Issue_category,Mô tả protocol,Mô tả nguyên nhân,Mô tả nhóm nguyên nhân,Nhóm Protocol
1,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,1.1.1.1,-,ds1,,-,G1,DVKH đào sâu vào nguyên nhân thực sự khiến khá...,Khách hàng chuyển đến địa chỉ mới có sẵn đường...,Nguyên nhân liên quan đến địa điểm,Đào sâu
3,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,1.1.2.1,-,ds3,,-,G1,DVKH hỏi khách hàng về nhu cầu sử dụng tại địa...,Khách hàng chuyển đến địa chỉ mới có sẵn đường...,Nguyên nhân liên quan đến địa điểm,Đào sâu
5,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,1.1.3.1,-,dx5,,-,G1,DVKH đề xuất khách hàng nhượng lại đường truyề...,Khách hàng chuyển đến địa chỉ mới có sẵn đường...,Nguyên nhân liên quan đến địa điểm,Đề xuất
7,1,1.2,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,1.2.1.1,-,hd5,,-,G3,DVKH hướng dẫn các vấn đề liên quan đến thanh ...,"Khách hàng không có bạn bè, người thân để chuy...",Nguyên nhân liên quan đến nhận thức dịch vụ,Hướng dẫn
9,2,2.1,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,2.1.1.1,-,ds1,,+,G0,DVKH đào sâu vào nguyên nhân thực sự khiến khá...,Khách hàng yêu cầu hỗ trợ nhưng chưa rõ lý do ...,Nguyên nhân liên quan đến quyết định chưa rõ ràng,Đào sâu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5869,684,684.2,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,684.2.2.1,-,ll3,,+,G5,Hợp đồng hiện tại của khách hàng có mức giá ưu...,Khách hàng đã/ sắp chuyển sang ISP khác vì lý ...,Nguyên nhân liên quan đến sự cạnh tranh từ ISP...,Lý lẽ
5870,684,684.2,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,684.2.2.2,+,ll5,,-,G5,"Là khách hàng lâu năm, khách hàng được hưởng c...",Khách hàng đã/ sắp chuyển sang ISP khác vì lý ...,Nguyên nhân liên quan đến sự cạnh tranh từ ISP...,Lý lẽ
5872,684,684.3,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,684.3.1.1,-,dx7,,+,G2,DVKH đề xuất trao đổi trực tiếp với người sử d...,KH không thường xuyên ở nhà,Nguyên nhân liên quan đến nhu cầu cá nhân,Đề xuất
5873,684,684.3,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,NaN,...,684.3.1.2,+,gt3,,+,G2,DVKH giải thích liên quan đến các gói cước hiệ...,KH không thường xuyên ở nhà,Nguyên nhân liên quan đến nhu cầu cá nhân,Giải thích


In [33]:
df[['Solution Result']].drop_duplicates('Solution Result')

,Solution Result
1,Không thành công
150,Lưu ý
239,Thành công
375,Cân nhắc


# get quote

In [34]:
def get_quote(df):
    processed_data = []
    
    for index, row in df.iterrows():
        quote_success = None
        quote_consideration = None
        quote_warning = None

        if row['Solution Result']:
            match row['Solution Result']:
                case "Thành công":
                    quote_success = row['Quote Success']

                case 'Cân nhắc':
                    for i in range(3):
                        if row['Solution'] == row[f'Indicate Consideration {i+1}']:
                            quote_consideration = row[f'Quote Consideration {i+1}']

                case 'Lưu ý':
                    for i in range(3):
                        if row['Solution'] == row[f'Indicate Warning {i+1}']:
                            quote_warning = row[f'Quote Warning {i+1}']

        # Copy the current row and update the 'Quote' column
        new_row = row.copy()
        if quote_success is not None:
            new_row['Quote of the result'] = quote_success
        elif quote_consideration is not None:
            new_row['Quote of the result'] = quote_consideration
        elif quote_warning is not None:
            new_row['Quote of the result'] = quote_warning

        
        processed_data.append(new_row)
    
    # Convert the processed data back into a DataFrame
    processed_df = pd.DataFrame(processed_data)
    
    return processed_df

In [35]:
df = get_quote(df)

In [36]:
df.notna().sum()

case_id                     3840
flow_id                     3840
DV_Số điện thoại            3840
DV_Số hợp đồng              3840
DV_Dịch vụ                  3840
TT_Vùng                     3840
TT_Giới tính                3806
TT_Nhóm tuổi                3814
TT_Mức độ am hiểu           3806
TT_Mục đích sử dụng         1316
TT_Thời gian sử dụng        3261
TT_Note                      933
KL_Nội dung hỗ trợ          3840
Flow                        3840
KL_Issue                     291
KL_Success                   608
KL_Warning                    58
KL_Consideration             259
KL_Đánh giá NV + đề xuất     380
Issue Code                  3840
Expand Issue                 209
Indicate Issue               145
Quote Issue                  128
Indicate Success            1335
Detail Success              1335
Quote Success                361
Indicate Warning 1            30
Detail Warning 1              30
Quote Warning 1               19
Indicate Warning 2             0
Detail War

In [37]:
label

,Mã,Mô tả,Unnamed: 2,Cách dùng
0,dx1,DVKH đề xuất kỹ thuật sang kiểm tra/bảo trì/kh...,NaN,NaN
1,dx2,"DVKH đề xuất ưu đãi, giảm giá",NaN,NaN
2,dx3,DVKH đề xuất tạm ngưng dịch vụ thay vì hủy hoà...,NaN,NaN
3,dx4,DVKH đề xuất hỗ trợ khách hàng di dời đường tr...,NaN,NaN
4,dx5,DVKH đề xuất khách hàng nhượng lại đường truyề...,NaN,NaN
...,...,...,...,...
108,gt19,Vấn đề khai thuế là trước đây truyền hình và n...,NaN,NaN
109,gt20,Các nhà mạng đã thống nhất không còn 6+1 chỉ c...,NaN,NaN
110,gt21,Nếu được sale (kể cả không phải của FPT)...,NaN,NaN
111,gt22,Giải thích lý do phải thu tiền chuyển địa điểm...,NaN,NaN


# some other processes

In [38]:
df.columns = df.columns.str.strip()

In [39]:
df['TT_Mục đích sử dụng'] = df['TT_Mục đích sử dụng'].replace({None: '-'})

In [40]:
df

,case_id,flow_id,DV_Số điện thoại,DV_Số hợp đồng,DV_Dịch vụ,TT_Vùng,TT_Giới tính,TT_Nhóm tuổi,TT_Mức độ am hiểu,TT_Mục đích sử dụng,...,Forward Cohesion,Protocol,Protocol Detail,Backward Cohesion,Issue_category,Mô tả protocol,Mô tả nguyên nhân,Mô tả nhóm nguyên nhân,Nhóm Protocol,Quote of the result
1,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,-,ds1,,-,G1,DVKH đào sâu vào nguyên nhân thực sự khiến khá...,Khách hàng chuyển đến địa chỉ mới có sẵn đường...,Nguyên nhân liên quan đến địa điểm,Đào sâu,NaN
3,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,-,ds3,,-,G1,DVKH hỏi khách hàng về nhu cầu sử dụng tại địa...,Khách hàng chuyển đến địa chỉ mới có sẵn đường...,Nguyên nhân liên quan đến địa điểm,Đào sâu,NaN
5,1,1.1,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,-,dx5,,-,G1,DVKH đề xuất khách hàng nhượng lại đường truyề...,Khách hàng chuyển đến địa chỉ mới có sẵn đường...,Nguyên nhân liên quan đến địa điểm,Đề xuất,NaN
7,1,1.2,0364222221,HNAAC5070,Giga,Hà Nội,Nữ,18 - 35,Có hiểu biết,-,...,-,hd5,,-,G3,DVKH hướng dẫn các vấn đề liên quan đến thanh ...,"Khách hàng không có bạn bè, người thân để chuy...",Nguyên nhân liên quan đến nhận thức dịch vụ,Hướng dẫn,NaN
9,2,2.1,0337762428,HNAAF0715,Sky,Hà Nội,Nam,18 - 35,Thấp,-,...,-,ds1,,+,G0,DVKH đào sâu vào nguyên nhân thực sự khiến khá...,Khách hàng yêu cầu hỗ trợ nhưng chưa rõ lý do ...,Nguyên nhân liên quan đến quyết định chưa rõ ràng,Đào sâu,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5869,684,684.2,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,-,...,-,ll3,,+,G5,Hợp đồng hiện tại của khách hàng có mức giá ưu...,Khách hàng đã/ sắp chuyển sang ISP khác vì lý ...,Nguyên nhân liên quan đến sự cạnh tranh từ ISP...,Lý lẽ,NaN
5870,684,684.2,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,-,...,+,ll5,,-,G5,"Là khách hàng lâu năm, khách hàng được hưởng c...",Khách hàng đã/ sắp chuyển sang ISP khác vì lý ...,Nguyên nhân liên quan đến sự cạnh tranh từ ISP...,Lý lẽ,NaN
5872,684,684.3,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,-,...,-,dx7,,+,G2,DVKH đề xuất trao đổi trực tiếp với người sử d...,KH không thường xuyên ở nhà,Nguyên nhân liên quan đến nhu cầu cá nhân,Đề xuất,Không biết mình có dùng zalo số này không. Bây...
5873,684,684.3,0987002109,SGJ080215,Giga,Hồ Chí Minh,Nữ,18 - 35,Cao,-,...,+,gt3,,+,G2,DVKH giải thích liên quan đến các gói cước hiệ...,KH không thường xuyên ở nhà,Nguyên nhân liên quan đến nhu cầu cá nhân,Giải thích,Không biết mình có dùng zalo số này không. Bây...


In [41]:
mother = df[['case_id', 'flow_id', 'solution_id', 'protocol_id', 'Flow',
         'DV_Số điện thoại', 'DV_Số hợp đồng', 'DV_Dịch vụ', 'TT_Vùng', 'TT_Giới tính', 'TT_Nhóm tuổi',
         'TT_Mức độ am hiểu', 'TT_Mục đích sử dụng', 'TT_Thời gian sử dụng', 'TT_Note', 'KL_Nội dung hỗ trợ', 
         'Issue Code', 'Issue_category', 'Mô tả nhóm nguyên nhân', 'Quote Issue',
         'Solution', 'Solution Result', 'Mô tả nguyên nhân',
         'Forward Cohesion', 'Protocol', 'Backward Cohesion', 'Protocol Detail', 'Mô tả protocol', 'Nhóm Protocol', 
         'Quote of the result'
        ]]

In [48]:
mother.to_excel('25Nov/mother of restructure V2.xlsx')

# Restructure

In [43]:
def generate_unique_ids(df, column_name='index', length=5):
    num_rows = len(df)
    unique_ids = random.sample(range(10**(length-1), 10**length), num_rows)
    
    df[column_name] = unique_ids
    return df

In [44]:
df = generate_unique_ids(df)

In [45]:
protocol_level = df[['index', 'case_id', 'flow_id',	'solution_id', 'protocol_id', 'Solution Result',
              'Forward Cohesion',	'Protocol',	'Backward Cohesion',
              'Protocol Detail', 'Mô tả protocol',	'Nhóm Protocol']].drop_duplicates()

solution_level = df[['case_id', 'flow_id', 'solution_id', 'Solution', 'Solution Result',
                     'Quote of the result']].drop_duplicates()

issue_level = df[['case_id', 'flow_id', 'Issue Code', 'Mô tả nguyên nhân',
                  'Issue_category', 'Mô tả nhóm nguyên nhân', 'Quote Issue']].drop_duplicates()

case_level = df[['case_id','DV_Số điện thoại',	'DV_Số hợp đồng', 'DV_Dịch vụ',
              'TT_Vùng',	'TT_Giới tính',	'TT_Nhóm tuổi', 'TT_Mức độ am hiểu',
              'TT_Mục đích sử dụng',	'TT_Thời gian sử dụng', 'TT_Note', 'KL_Nội dung hỗ trợ']].drop_duplicates()

flow_level = df[['case_id', 'flow_id',	'Flow']].drop_duplicates()

In [46]:
output_file = '25Nov\Restructure_v6_25Nov.xlsx'

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    protocol_level.to_excel(writer, sheet_name='protocol_level', index=False)
    solution_level.to_excel(writer, sheet_name='solution_level', index=False)
    issue_level.to_excel(writer, sheet_name='issue_level', index=False)
    case_level.to_excel(writer, sheet_name='case_level', index=False)
    flow_level.to_excel(writer, sheet_name='flow_level', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\R'
<>:1: SyntaxWarning: invalid escape sequence '\R'
C:\Users\anhthy\AppData\Local\Temp\ipykernel_26220\689331980.py:1: SyntaxWarning: invalid escape sequence '\R'
  output_file = '25Nov\Restructure_v6_25Nov.xlsx'


In [47]:
flow_level

,case_id,flow_id,Flow
1,1,1.1,n12=s1>ds1\s2>ds3\s3>dx5
7,1,1.2,n34=s1>hd5
9,2,2.1,n00=s1>ds1+dx4
12,2,2.2,n10=s1>ds3\s2>ds1\s3>ll3+dx5
19,2,2.3,n34=s1>hd1+hd5
...,...,...,...
5855,683,683.2,n21[truyền hình]=s1'>ll2+ds2
5858,683,683.3,n20[internet]=s1'>gt3+dx9[combo giga cộng tru...
5862,684,684.1,n21#[truyền hình]=s1!>ll1+ll2+ll16+gt1
5867,684,684.2,n50=s1>ds1\s2>ll3+ll5>s3>dx10[6+1 hoặc 12+2]
